In [4]:
import re
import os
from tqdm import tqdm
import shutil
from os.path import join
from datetime import datetime

def rm_s(stem:str) :
    "반복문 처음(stem)과 끝(name,extra)에서 실행하라. 언더봐와 스페이스 > strip된 스페이스"
    #시작,중간,끝 모든 공백과 _를 > 하나의 공백으로 > strip하면 끝
    p_s = re.compile("^[_\s)]+|[_\s(]+$|[_\s]{1,}") #(로 끝나거나 )로 시작하는 경우는 제거. 짝 안 맞는 건 주)가 있으니 그냥 두자..
    stem = p_s.sub(" ", stem).strip()
    return stem

def rmNeedless(extra:str) :
    """name과 extra가 확정되면 그때 한 번 해준다. 공백으로 대체가 있으므로 rm_s()를 내포했다.
    일련번호와 기호 제거"""
       
    #완전제거                                                      영어와 숫자가 연속
    p_rmSerialN = re.compile(r"(?<![a-zA-Z])[a-zA-Z](?![a-zA-Z])|\d+[a-zA-Z]+|[a-zA-Z]+\d+|TAA\(회\)|\
        |SCSB|ADMIN.*Conflict|\d(?!건|통|차|채|자|억|천|백|급|번|길|명)") # 모든 숫자를 지운다. 해당글자가 나온다면 그 앞 숫자는 살린다. 
    p_sign = re.compile(r"[^#㈜()\sa-zA-Zㄱ-ㅎ가-힣\d]|\([^\w]*\)") #반쪽 괄호만 있는 거는 어케 지우지?
    
    extra = p_rmSerialN.sub("", extra)
    extra = p_sign.sub(" ", extra)

    extra = rm_s(extra)

    return extra

def re_name(src: str, dst: str) -> list:
    """
    파일명 변경 후 os.rename대신 사용하는 함수(중복확인과 새넘버링)
    폴더를 바꾸는 거 아니라면 파일명 다를때라는 조건문 다음에 호출
    src : dir + file
    dst : dir + new_name
    return : [src dir, src filename, dst filename, dst dir]
    """
    dir = os.path.split(dst)[0]
    f_name = os.path.split(dst)[1]
    stem = os.path.splitext(f_name)[0]
    ext = os.path.splitext(f_name)[1]

    new_name = stem + ext
    i = 1
    while os.path.exists(dir+"/"+new_name):  # 작업디렉토리가 아니므로 풀경로
        new_name = stem + "_"+"("+str(i)+")"+ext
        i += 1

    if not os.path.exists(dir):
        os.makedirs(dir)  # 미리 만들어뒀으니 mkdir해도 됨
    
    shutil.move(src, join(dir, new_name))

    return [os.path.split(src)[0],os.path.split(src)[1], new_name, dir]

def write_log_csv(log:list, path : str) -> None:
    "2차원이 최적이고 그 이상은 셀안에 리스트로"
    if not os.path.exists(path):
        os.makedirs(path)  

    if len(log) > 0 :
    
        import csv
        
        name = str(datetime.today().strftime("%Y%m%d %H%M%S")) + "_" +str(len(log)) + ".csv" 
                    
        with open (join(path, name), "a", newline="", encoding='utf-8-sig') as p :
            for row in log :
                wr = csv.writer(p)
                wr.writerow(row)
    else :
        print(f'log 파일이 비어있습니다. path : {path}')



p = re.compile("ADMIN.*Conflict")
p1 = re.compile("[A-Z]")
path = r"C:\Users\SL\Desktop\test"
path_log_success = path
cnt = 0
ischanged = False
success = []
for root, __dirs__, files in tqdm(os.walk(path)) :
    for f in files :
        stem, ext = os.path.splitext(f)
        if p.search(stem) :
            stem = p.sub("", stem)
            ischanged = True
        if p1.search(ext) :
            ext = ext.lower()
            ischanged = True
        
        if ischanged :
            stem = rmNeedless(stem)
            stem = rm_s(stem)
            new_name = stem + ext
            success.append(re_name(join(root, f), join(root, new_name)))
        
write_log_csv(success, path_log_success)


            
print(cnt)



1it [00:00, 250.84it/s]

0


In [1]:
import pickle # 내장모듈
from os.path import join
from tqdm import tqdm
import os
import pandas as pd

In [2]:
### 보고서용 폴더 트리
pathOfDict = r"D:\0.전산\1.진행중과업\파일서버개편\파일\중복조사"
nameOfDict = "정렬된 sc별 파일정보.pkl"

with open(join(pathOfDict, nameOfDict), "rb") as pkl :
    dict = pickle.load(pkl)
total = len(dict)
print(total) 

991093


In [22]:
keys, extraKeys, remains = [], [], []
sep = os.path.sep
for __key__, list in tqdm(dict.items(), total=total) :
    file, temp, local_dir, extra_dir = None, None, None, None
    src = list[0]["fullPath"]
    dir = os.path.split(src)[0]

    file = os.path.split(src)[1] #
    temp = dir.split(sep, maxsplit=3)
    local_dir = temp[2] #
    if len(temp) > 3 :
        extra_dir = temp[3] #
    
    if list[0]["score"] >= 100000 :
        keys.append([file, local_dir, extra_dir])
    elif list[0]["score"] >= 10000 :
        extraKeys.append([file, local_dir, extra_dir])
    else :
        remains.append([file, local_dir, extra_dir])
print(len(keys), len(extraKeys), len(remains))

100%|██████████| 991093/991093 [00:05<00:00, 165958.03it/s]

201317 324387 465389


In [18]:
print(keys[13100])

['20419188_우미화_원인서류.pdf', '서울', 'OK캐피탈\\OK캐피탈2차']


In [23]:
columns = ["file", "local_dir", "extra_dir"]
key_df = pd.DataFrame(keys, columns=columns)
extraKey_df = pd.DataFrame(extraKeys, columns=columns)
remain_df = pd.DataFrame(remains, columns=columns)
print(len(key_df), len(extraKey_df), len(remain_df))

201317 324387 465389


In [24]:
save_path = r'D:\0.전산\1.진행중과업\파일서버개편\폴더트리 보고서'
key_df.to_excel(join(save_path,"key.xlsx"), encoding='utf-8-sig', index=False)
extraKey_df.to_excel(join(save_path,"extraKey.xlsx"), encoding='utf-8-sig', index=False)
remain_df.to_excel(join(save_path,"remain.xlsx"), encoding='utf-8-sig', index=False)

In [2]:
########### 파일 지운 거 다시 복구
import os
import shutil
import time
import pandas as pd
from tqdm import tqdm      #tqdm(filelist, total = len(file_list), position=0, leave=True)
import re
from pathlib import Path
import traceback
from os.path import join
import file_function as ff
from datetime import datetime
from upload_v2 import dict_refer, write_log_csv, re_name
import pickle

path_log = r'D:\0.전산\1.진행중과업\파일서버개편\log'

In [29]:
df = pd.read_excel(r'D:\0.전산\1.진행중과업\파일서버개편\폴더트리 보고서\extraKey.xlsx', sheet_name=[1])

In [30]:
df = df[1]

In [32]:
data = df.query('local_dir == "광주 공유폴더"' and 'extra_dir == "모음"' and '일 == "대출원장"' and '이 == "케이에스자산관리"')

In [33]:
data

,file,local_dir,extra_dir,일,이,삼,사,오,육,칠
4823,#1_5610152690319_ANY_TYM_001.tif,광주 공유폴더,모음,대출원장,케이에스자산관리,펜타원인서류,6001001184,NaN,NaN,NaN
13775,N2_5604152551114_TYM_PENCO_001.pdf,광주 공유폴더,모음,대출원장,케이에스자산관리,펜타원인서류,6001000015,NaN,NaN,NaN
13776,#1_5708111530315_ANY_TYM_001.tif,광주 공유폴더,모음,대출원장,케이에스자산관리,펜타원인서류,6001000022,NaN,NaN,NaN
13777,M2_5708111530315_TYM_PENCO_001.pdf,광주 공유폴더,모음,대출원장,케이에스자산관리,펜타원인서류,6001000022,NaN,NaN,NaN
13778,N2_3901262530326_TYM_PENCO_001.pdf,광주 공유폴더,모음,대출원장,케이에스자산관리,펜타원인서류,6001000022,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
21150,08627_2002_476_4_08_000000000000000_3.tif,광주 공유폴더,모음,대출원장,케이에스자산관리,펜타원인서류,9009000936,NaN,NaN,NaN
21151,08627_2002_476_4_08_000000000000000_4.tif,광주 공유폴더,모음,대출원장,케이에스자산관리,펜타원인서류,9009000936,NaN,NaN,NaN
21152,08627_2002_476_A_01_000000000000000_1.tif,광주 공유폴더,모음,대출원장,케이에스자산관리,펜타원인서류,9009000936,NaN,NaN,NaN
21153,08627_2002_476_B_09_000000000000000_1.tif,광주 공유폴더,모음,대출원장,케이에스자산관리,펜타원인서류,9009000936,NaN,NaN,NaN


In [36]:
src_base = r"\\192.168.0.75\스캔파일\구 스캔파일"
dst_base = r"D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류"
log = []
fail = []
for i, v in tqdm(data.iterrows()) :
    src = join(src_base, v.local_dir, v.extra_dir, v.일, v.이, v.삼, str(v.사), v.file)
    new_name = str(v.사) + " " + v.file
    dst = join(dst_base, new_name)
    try : 
        shutil.copy2(src, dst)
        log.append([src, dst])
    except Exception as e :
        fail.append([src, dst, e.__class__, e])

print(f"{len(log)}개 파일 복사.  {len(fail)}개 파일 복사 실패")
write_log_csv(log, dst_base, "성공")
write_log_csv(fail, dst_base, "실패")
    

7381it [20:51,  5.90it/s]

7381개 파일 복사.  0개 파일 복사 실패
log 파일이 비어있습니다. path : D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류


In [41]:
test = pd.read_csv(r"D:\0.전산\1.진행중과업\파일서버개편\log\20221026 105812_4523_광주 모음 대출원장 케이에스 펜타.csv", header=None)
test.columns = ["dir", "file", "일", "이"]
test

,dir,file,일,이
0,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류,6001000084 N2_7609172067438_TYM_PENCO_001.pdf,20414562 6001000084 N2_7609172067438_TYM_PENCO...,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류
1,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류,6001000087 #1_4178132771_ANY_TYM_001.tif,20411220 6001000087 #1_4178132771_ANY_TYM_001.tif,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류
2,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류,6001000087 #2_4178132771_ANY_TYM_001.tif,20411220 6001000087 #2_4178132771_ANY_TYM_001.tif,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류
3,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류,6001000087 206007100540669100_김막순.tif,20411220 6001000087 206007100540669100_김막순.tif,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류
4,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류,6001000087 206007100540669100_윤두남.tif,20411220 6001000087 206007100540669100_윤두남.tif,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류
...,...,...,...,...
4518,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류,9009000936 08627_2002_476_4_08_000000000000000...,20429422 9009000936 08627_2002_476_4_08_000000...,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류
4519,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류,9009000936 08627_2002_476_4_08_000000000000000...,20429422 9009000936 08627_2002_476_4_08_000000...,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류
4520,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류,9009000936 08627_2002_476_A_01_000000000000000...,20429422 9009000936 08627_2002_476_A_01_000000...,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류
4521,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류,9009000936 08627_2002_476_B_09_000000000000000...,20429422 9009000936 08627_2002_476_B_09_000000...,D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류


In [46]:
files = list(test.file)

In [49]:
files[0][11:]

'N2_7609172067438_TYM_PENCO_001.pdf'

In [60]:
error = []
for f in files :
    if any(data.file.str.extract(f[11:])) :
        pass
    else :
        error.append(f)
print(len(error))

C:\Users\SL\AppData\Local\Temp\ipykernel_43944\2222372003.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if any(data.file.str.contains(f[11:])) :


720


In [61]:
error

['6001000087 206007100540669100_김막순.tif',
 '6001000087 206007100540669100_윤두남.tif',
 '6001000483 005905060674400100,32.PDF',
 '6001000694 02445_2002_4302_B_01_000000000000000.tif',
 '6001000694 02445_2002_4302_B_29_000000000000000.tif',
 '6001000694 02445_2002_4302_D_02_000000000000000.tif',
 '6001000694 02445_2002_4302_D_03_000000000000000.tif',
 '6001000694 02445_2002_4302_D_06_000000000000000.tif',
 '6001000707 09252_2000_36298_4_02_000000000000000.tif',
 '6001000707 09252_2000_36298_4_08_000000000000000.tif',
 '6001000707 09252_2000_36298_4_08_000000000000000_(7).tif',
 '6001000707 09252_2000_36298_4_08_000000000000000_(8).tif',
 '6001000707 09252_2000_36298_4_08_000000000000000_(9).tif',
 '6001000707 09252_2000_36298_B_01_000000000000000.tif',
 '6001000707 09252_2000_36298_B_02_000000000000000.tif',
 '6001000707 09252_2000_36298_B_99_000000000000000.tif',
 '6001000707 09252_2000_36298_D_03_000000000000000.tif',
 '6001000707 09252_2000_36298_D_05_000000000000000.tif',
 '6001000707 